In [1]:
import dataset

import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

import torch
import torch.nn as nn
from torch import Tensor
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from sklearn.model_selection import train_test_split

In [2]:
random_seed = 11

torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True # 고정하면 학습이 느려진다고 합니다.
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
np.random.seed(random_seed)
random.seed(random_seed)


In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


In [ ]:
# params
target = 'age'
MASK_CLASS_NUM = 3
LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 10 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션
loss_fn = torch.nn.CrossEntropyLoss()

In [4]:
df = pd.read_csv('../input/data/train/train_new.csv')
train_df, valid_df = train_test_split(df, test_size=0.2, stratify = df[target], random_state=11)

In [5]:

transform = dataset.BaseAugmentation()
train_data = dataset.MaskDataset(train_df,target,transform)
valid_data = dataset.MaskDataset(valid_df,target,transform)
train_iter = DataLoader(train_data,batch_size=64,shuffle=True)
valid_iter = DataLoader(valid_data,batch_size=128,shuffle=True)


In [6]:
transform_aug = dataset.CustomAugmentation()
train_data_aug = dataset.MaskDataset(train_df,target,transform_aug)
train_iter_aug = DataLoader(train_data_aug,batch_size=64,shuffle=True)

In [7]:
import math
# ImageNet에서 학습된 ResNet 18 딥러닝 모델을 불러옴
imagenet_resnet18 = torchvision.models.resnet18(pretrained=True)

# 분류 모델의 output 크기가 1000개로 되어 있음으로 mnist data class 개수로 나올 수 있도록 Fully Connected Layer를 변경하고 xavier uniform으로 weight 초기화
imagenet_resnet18.fc = torch.nn.Linear(in_features=512, out_features=MASK_CLASS_NUM, bias=True)
torch.nn.init.xavier_uniform_(imagenet_resnet18.fc.weight)
stdv = 1. / math.sqrt(imagenet_resnet18.fc.weight.size(1))
imagenet_resnet18.fc.bias.data.uniform_(-stdv, stdv)

print("네트워크 필요 입력 채널 개수", imagenet_resnet18.conv1.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", imagenet_resnet18.fc.weight.shape[0])

네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 3


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

imagenet_resnet18.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함
optimizer = torch.optim.Adam(imagenet_resnet18.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

cuda:0 is using!


In [ ]:
from tqdm import tqdm

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(1,NUM_EPOCH+1):
    for phase in ["train", "test"]:
            running_loss = 0.
            running_acc = 0.

  ###############################################################################################################
    if phase == "train":
        imagenet_resnet18.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        for ind, (a,b) in enumerate(tqdm(zip(train_iter,train_iter_aug))):
                images = torch.cat((a[0],b[0])).to(device)
                labels = torch.cat((a[1],b[1])).to(device)

                optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

                with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                    logits = imagenet_resnet18(images)
                    _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
                    loss = loss_fn(logits, labels)

                    loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
                    optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

                running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
                running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
        
        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / (len(train_iter.dataset) + len(train_iter_aug.dataset))
        epoch_acc = running_acc / (len(train_iter.dataset) + len(train_iter_aug.dataset))
    
    elif phase == "test":
        imagenet_resnet18.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함
        for ind,a in enumerate(tqdm(valid_iter)):
                images = a[0].to(device)
                labels = a[1].to(device)

                optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

                with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                    logits = imagenet_resnet18(images)
                    _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
                    loss = loss_fn(logits, labels)

                running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
                running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(valid_iter.dataset)
        epoch_acc = running_acc / len(valid_iter.dataset)
  ############################################################################################################################

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
        best_test_accuracy = epoch_acc
        torch.save(imagenet_resnet18.state_dict(), f'./model/{target}_state_dict/{epoch:03d}_{epoch_acc*100%100:03.2f}.pt')
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
        best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

In [10]:
torch.save(imagenet_resnet18.state_dict(), f'./model/{target}.pt')